# Binary Search Trees
A BST is a binary tree in symmetric order.

**Binary Tree**: a binary tree is either
* _empty_ or
* _two disjoint binary trees_ (left and right)

**Symmetric Order**: Each node has a key, and every node's key is:
* larger than all keys in its left subtree
* smaller than all keys in its right subtree

**Searching**:
* if less, go left.
* if greater, go right.
* if equal, search hit.

**Unsuccessful search**:
* finds a left or right reference

**Insertion**: same for unsuccessful search, but create a node there.

**Tree shape**: depends on order of insertion. Worst case is like a linked list, where we've inserted items in sorted order

References:
* https://algs4.cs.princeton.edu/32bst/
* OpenCourseware: https://www.youtube.com/watch?v=76dhtgZt38A&list=PLUl4u3cNGP63EdVPNLG3ToM6LaEUuStEY&index=9
* Implementation: https://gist.github.com/zjplab/b052af68196bec5ce48dcc39b0acef2f

In [3]:
# a binary search tree is just a reference to a root Node
# each Node has four fields:
# a key
# a value
# references to a left and right Node
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.left = None
        self.right = None

In [4]:
def get(node: Node, key: str):
    """Returns the value from the given BST, or None
    Cost: 1 + depth of the tree
    """
    if not node:
        return None
    if key < node.key:
        return get(node.left, key)
    elif key > node.key:
        return get(node.right, key)
    else:
        return node.value

In [5]:
def insert(node: Node, key: str, value: str) -> Node:
    """Adds the key and value to the tree
    If the key is already present, change the current value
    to the new value.
    """
    if not node:
        return Node(key, value)

    if key == node.key:
        node.value = value
    elif key < node.key:
        node.left = insert(node.left, key, value)
    else:
        node.right = insert(node.right, key, value)

    return node

Note: Correspondence between BSTs and quicksort partitioning. If no duplicate keys, there's a 1-to-1 between what happens in a BST and in quicksort. For a given key, everything less than is to the left, everything greater than is to the right.

Proposition: If N distinct keys are inserted into a BST in _random_ order, the expected number of compares for a search/insert is ~2 ln N.
Pf. 1-to-1 correspondence with quicksort partitioning.

In [6]:
def min_node(node):
    """Returns the minimum value in the tree."""
    current = node
    while current.left is not None:
        current = current.left
    return current

def max_node(node):
    """Returns the maximum value in the tree."""
    current = node
    while current.right is not None:
        current = current.right
    return current

In [7]:
def floor(root, key):
    """Find the largest key <= a given key.
    Case 1: k equals the key at root -> floor(k) = k
    Case 2: k is less than the key at the root -> floor(k) is in the left subtree
    Case 3: k is greater than the key at the root -> floor(k) is in the right subtree,
    if there is _any_ key <= k in the right subtree, otherwise it is the key in the root."""
    if not root:
        return
    if key == root.key:
        return key
    elif key < root.key:
        if root.left:
            return floor(root.left, key)
        else:
            return None
    else:
        if root.right:
            k = floor(root.right, key)
            return k if k else root.key
        else:
            return root.key

The put function above is iterative. If we need to keep track of how many nodes are below a certain node, a recursive implementation is probably easier. Same if we need to answer how many keys are < k?

# Traversals

## Iterating over the tree: Inorder traversal
1. Traverse left subtree
2. enqueue key
3. Traverse right subtree

In [8]:
from collections import deque

def traverse_inorder(node):
    q = deque()
    _traverse_inorder(node, q)
    return q

def _traverse_inorder(node, q):
    if not node:
        return None
    _traverse_inorder(node, q.left)
    q.append(node.key)
    _traverse_inorder(node, q.right)

# BST deletion


In [9]:
def delete_node(root, key):

    # Base Case
    if root is None:
        return root

    # If the key to be deleted is smaller than the root's
    # key, then it lies in left subtree
    if key < root.key:
        root.left = delete_node(root.left, key)

    # If the key to be deleted is greater than the root's key,
    # then it lies in right subtree
    elif key > root.key:
        root.right = delete_node(root.right, key)

    # If key is same as root's key, then this is the node
    # to be deleted
    else:
        # Node with only one child or no child
        if root.left is None :
            temp = root.right
            root = None
            return temp

        elif root.right is None :
            temp = root.left
            root = None
            return temp

        # Node with two children: Get the inorder successor
        # (smallest in the right subtree)
        temp = min_node(root.right)

        # Copy the inorder successor's content to this node
        root.key = temp.key

        # Delete the inorder successor
        root.right = delete_node(root.right , temp.key)

    return root

## General deletion
To delete a node with key _k_: search for node _n_ containing key _k_.
Case 0: [0 Children] Delete _n_ by setting parent link to null
Case 1: [1 child] Delete _n_ by setting parent link to that child
Case 2: [2 children]
    1. Find smallest node in right subtree. So, go right once, then left until you hit a null link.
    2. Delete the minimum in _n_'s right subtree
    3. Put _x_ in _n_'s spot
    4. update links and node counts after recursive calls

In [10]:
# not good
def delete(node, key):
    if not node:
        return None
    if key < node.key:                     # Searching for the key
        delete(node.left, key)             # |
    elif key > node.key:                   # |
        delete(node.right, key)            # |
    else:                                  # Found it
        if not node.right:                 # Cases 0 and 1: 0 or 1 children
            return node.left               # |
        if not node.left:                  # |
            return node.right              # |
                                           # |
        temp = node                        # Case 2: 2 children
        node = min(temp.right)             # |
        node.right = delete_min_key(temp.right)
        node.left = temp.left              # |

    node.count = node.left.size + node.right.size + 1
    return node

The issue with the above implementation is that, over time, the tree becomes less balanced. The height of the tree will eventually become sqrt(N) rather than lg N. Random choice between subtrees shows the same problem.